In [2]:
import pandas as pd
from pathlib import Path

# 1. Load the raw datasets
df_full = pd.read_csv("data/raw_data.csv")
df_inc = pd.read_csv("data/incremental_data.csv")

# Helper: show before/after transformations
def show_diff(before, after, message):
    print(f"\n--- {message} ---")
    print("Before:")
    display(before.head())
    print("After:")
    display(after.head())
    

In [4]:
# a. Cleaning
# # Fill missing numeric fields with median
before_full = df_full.copy(deep=True)
before_inc = df_inc.copy(deep=True)
num_cols = df_full.select_dtypes(include='number').columns
for col in num_cols:
    median = df_full[col].median()
    df_full[col] = df_full[col].fillna(median)
    if col in df_inc.columns:
        df_inc[col] = df_inc[col].fillna(median)

# Fill missing categorical fields with mode
cat_cols = df_full.select_dtypes(exclude='number').columns
for col in cat_cols:
    mode = df_full[col].mode()
    fill_value = mode[0] if not mode.empty else ""
    df_full[col] = df_full[col].fillna(fill_value)
    if col in df_inc.columns:
        df_inc[col] = df_inc[col].fillna(fill_value)

show_diff(before_full, df_full, "Filled missing values in FULL data")
show_diff(before_inc, df_inc, "Filled missing values in INCREMENTAL data")



--- Filled missing values in FULL data ---
Before:


,order_id,customer_name,product,quantity,unit_price,order_date,region
0,1.0,Linda,Isuzu,10.0,19410.0,12/25/2023,Tanzania
1,505.0,Lynda,Lexus,2.0,20207.0,6/13/2023,Tanzania
2,505.0,Emlyn,Chevrolet,10.0,20045.0,8/17/2023,Kenya
3,4.0,Sheila,Volvo,10.0,20207.0,2/26/2023,Tanzania
4,505.0,Elbert,Dodge,2.0,24052.0,4/11/2023,Uganda


After:


,order_id,customer_name,product,quantity,unit_price,order_date,region
0,1.0,Linda,Isuzu,10.0,19410.0,12/25/2023,Tanzania
1,505.0,Lynda,Lexus,2.0,20207.0,6/13/2023,Tanzania
2,505.0,Emlyn,Chevrolet,10.0,20045.0,8/17/2023,Kenya
3,4.0,Sheila,Volvo,10.0,20207.0,2/26/2023,Tanzania
4,505.0,Elbert,Dodge,2.0,24052.0,4/11/2023,Uganda



--- Filled missing values in INCREMENTAL data ---
Before:


,order_id,customer_name,product,quantity,unit_price,order_date
0,1001.0,Leila,Lincoln,3.0,28114,2025-01-09
1,505.0,Ranna,Isuzu,4.0,12801,1/1/2023
2,1003.0,Margarete,Mazda,9.0,16058,1/1/2023
3,1004.0,Franzen,Jeep,6.0,23892,2025-10-04
4,505.0,Prentice,Mitsubishi,6.0,15834,1/1/2023


After:


,order_id,customer_name,product,quantity,unit_price,order_date
0,1001.0,Leila,Lincoln,3.0,28114,2025-01-09
1,505.0,Ranna,Isuzu,4.0,12801,1/1/2023
2,1003.0,Margarete,Mazda,9.0,16058,1/1/2023
3,1004.0,Franzen,Jeep,6.0,23892,2025-10-04
4,505.0,Prentice,Mitsubishi,6.0,15834,1/1/2023


In [5]:
# b. ENRICHMENT: Add total_price = quantity * unit_price 
if {'quantity', 'unit_price'}.issubset(df_full.columns):
    df_full['total_price'] = df_full['quantity'] * df_full['unit_price']
    df_inc['total_price'] = df_inc['quantity'] * df_inc['unit_price']
    show_diff(before_full, df_full, "Added total_price column in FULL data")
    show_diff(before_inc, df_inc, "Added total_price column in INCREMENTAL data")


--- Added total_price column in FULL data ---
Before:


,order_id,customer_name,product,quantity,unit_price,order_date,region
0,1.0,Linda,Isuzu,10.0,19410.0,12/25/2023,Tanzania
1,505.0,Lynda,Lexus,2.0,20207.0,6/13/2023,Tanzania
2,505.0,Emlyn,Chevrolet,10.0,20045.0,8/17/2023,Kenya
3,4.0,Sheila,Volvo,10.0,20207.0,2/26/2023,Tanzania
4,505.0,Elbert,Dodge,2.0,24052.0,4/11/2023,Uganda


After:


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price
0,1.0,Linda,Isuzu,10.0,19410.0,12/25/2023,Tanzania,194100.0
1,505.0,Lynda,Lexus,2.0,20207.0,6/13/2023,Tanzania,40414.0
2,505.0,Emlyn,Chevrolet,10.0,20045.0,8/17/2023,Kenya,200450.0
3,4.0,Sheila,Volvo,10.0,20207.0,2/26/2023,Tanzania,202070.0
4,505.0,Elbert,Dodge,2.0,24052.0,4/11/2023,Uganda,48104.0



--- Added total_price column in INCREMENTAL data ---
Before:


,order_id,customer_name,product,quantity,unit_price,order_date
0,1001.0,Leila,Lincoln,3.0,28114,2025-01-09
1,505.0,Ranna,Isuzu,4.0,12801,1/1/2023
2,1003.0,Margarete,Mazda,9.0,16058,1/1/2023
3,1004.0,Franzen,Jeep,6.0,23892,2025-10-04
4,505.0,Prentice,Mitsubishi,6.0,15834,1/1/2023


After:


,order_id,customer_name,product,quantity,unit_price,order_date,total_price
0,1001.0,Leila,Lincoln,3.0,28114,2025-01-09,84342.0
1,505.0,Ranna,Isuzu,4.0,12801,1/1/2023,51204.0
2,1003.0,Margarete,Mazda,9.0,16058,1/1/2023,144522.0
3,1004.0,Franzen,Jeep,6.0,23892,2025-10-04,143352.0
4,505.0,Prentice,Mitsubishi,6.0,15834,1/1/2023,95004.0


In [6]:
# c. STRUCTURAL: Convert date columns to datetime 
date_cols = [col for col in df_full.columns if 'date' in col.lower()]
for col in date_cols:
    df_full[col] = pd.to_datetime(df_full[col], errors='coerce')
    df_inc[col] = pd.to_datetime(df_inc[col], errors='coerce')
    show_diff(before_full, df_full, f"Converted {col} to datetime in FULL data")
    show_diff(before_inc, df_inc, f"Converted {col} to datetime in INCREMENTAL data")


--- Converted order_date to datetime in FULL data ---
Before:


,order_id,customer_name,product,quantity,unit_price,order_date,region
0,1.0,Linda,Isuzu,10.0,19410.0,12/25/2023,Tanzania
1,505.0,Lynda,Lexus,2.0,20207.0,6/13/2023,Tanzania
2,505.0,Emlyn,Chevrolet,10.0,20045.0,8/17/2023,Kenya
3,4.0,Sheila,Volvo,10.0,20207.0,2/26/2023,Tanzania
4,505.0,Elbert,Dodge,2.0,24052.0,4/11/2023,Uganda


After:


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price
0,1.0,Linda,Isuzu,10.0,19410.0,2023-12-25,Tanzania,194100.0
1,505.0,Lynda,Lexus,2.0,20207.0,2023-06-13,Tanzania,40414.0
2,505.0,Emlyn,Chevrolet,10.0,20045.0,2023-08-17,Kenya,200450.0
3,4.0,Sheila,Volvo,10.0,20207.0,2023-02-26,Tanzania,202070.0
4,505.0,Elbert,Dodge,2.0,24052.0,2023-04-11,Uganda,48104.0



--- Converted order_date to datetime in INCREMENTAL data ---
Before:


,order_id,customer_name,product,quantity,unit_price,order_date
0,1001.0,Leila,Lincoln,3.0,28114,2025-01-09
1,505.0,Ranna,Isuzu,4.0,12801,1/1/2023
2,1003.0,Margarete,Mazda,9.0,16058,1/1/2023
3,1004.0,Franzen,Jeep,6.0,23892,2025-10-04
4,505.0,Prentice,Mitsubishi,6.0,15834,1/1/2023


After:


,order_id,customer_name,product,quantity,unit_price,order_date,total_price
0,1001.0,Leila,Lincoln,3.0,28114,2025-01-09,84342.0
1,505.0,Ranna,Isuzu,4.0,12801,NaT,51204.0
2,1003.0,Margarete,Mazda,9.0,16058,NaT,144522.0
3,1004.0,Franzen,Jeep,6.0,23892,2025-10-04,143352.0
4,505.0,Prentice,Mitsubishi,6.0,15834,NaT,95004.0


In [7]:
# Saving transformed files
Path("transformed").mkdir(exist_ok=True)
df_full.to_csv("transformed/transformed_full.csv", index=False)
df_inc.to_csv("transformed/transformed_incremental.csv", index=False)
print("\nTransformed files saved in 'transformed/' folder.")


Transformed files saved in 'transformed/' folder.
